In [3]:
from openai import AzureOpenAI
import dotenv
import os

# Load environment variables from .env file
dotenv.load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4-turbo", #Replace with model deployment name
  tools=[{
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  }, {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    } 
  }]
)

In [4]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_MfnjYyCkMcFDUs3Fhy6KjRCf",
  "created_at": 1709643692,
  "description": null,
  "file_ids": [],
  "instructions": "You are a weather bot. Use the provided functions to answer questions.",
  "metadata": {},
  "model": "gpt-4-turbo",
  "name": null,
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "description": "Get the weather in location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state e.g. San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": [
                "c",
                "f"
              ]
            }
          },
          "required": [
            "location"
          ]
        }
      },
      "type": "function"
    },
    {
      "function": {
        "name": "getNickname",
        "description": "Get 

In [6]:
thread = client.beta.threads.retrieve("thread_MOdoVYJd9elDyIVCtogUtbnX")
print(thread.model_dump_json(indent=2))

{
  "id": "thread_MOdoVYJd9elDyIVCtogUtbnX",
  "created_at": 1709642433,
  "metadata": {},
  "object": "thread"
}


In [10]:
assistant = client.beta.assistants.retrieve("asst_MfnjYyCkMcFDUs3Fhy6KjRCf")

In [11]:
thread = client.beta.threads.retrieve("thread_MOdoVYJd9elDyIVCtogUtbnX")
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_4fFzZuufxrQe2biicOJYQYeh",
      "assistant_id": "asst_FuM7QE8yaou7RyZPzAQ6pSZH",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Is there anything else you would like to visualize or any other modifications to this visualization?"
          },
          "type": "text"
        }
      ],
      "created_at": 1709642848,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_MiEZm6ApDeOTezHjhL5Ytohn",
      "thread_id": "thread_MOdoVYJd9elDyIVCtogUtbnX"
    },
    {
      "id": "msg_pNUg7BIsL2zCke1nKLZmO57n",
      "assistant_id": "asst_FuM7QE8yaou7RyZPzAQ6pSZH",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Is there anything else you would like to visualize or any other modifications to this visualization?"
          },
          "type": "text"
        }
      ],
      "cr

In [36]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="what is the weather in San Francisco, CA?"
)

TypeError: Messages.create() got an unexpected keyword argument 'tool_choice'

In [13]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_TqDBQ9VZNG8NuvtFcRQpWyCn",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "what is the weather in San Francisco, CA?"
          },
          "type": "text"
        }
      ],
      "created_at": 1709645103,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_MOdoVYJd9elDyIVCtogUtbnX"
    },
    {
      "id": "msg_4fFzZuufxrQe2biicOJYQYeh",
      "assistant_id": "asst_FuM7QE8yaou7RyZPzAQ6pSZH",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Is there anything else you would like to visualize or any other modifications to this visualization?"
          },
          "type": "text"
        }
      ],
      "created_at": 1709642848,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "a

In [14]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [15]:
import time
from IPython.display import clear_output


def wait_for_response(thread, run):
  start_time = time.time()

  status = run.status

  while status not in ["completed", "cancelled", "expired", "failed"]:
      time.sleep(5)
      run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
      print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
      status = run.status
      print(f'Status: {status}')
      clear_output(wait=True)

  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  ) 

  print(f'Status: {status}')
  print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
  print(messages.model_dump_json(indent=2))

In [33]:
import json
run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
print((run.id, run.status))
print(json.loads(run.required_action.model_dump_json(indent=2))['submit_tool_outputs']['tool_calls'])
call_id = json.loads(run.required_action.model_dump_json(indent=2))['submit_tool_outputs']['tool_calls'][0]["id"]
print(call_id)

('run_TLq4wtYjVQwbOTXNMJJ2MnFB', 'requires_action')
[{'id': 'call_y9Kp1zTC2steV8JskBTVx0QG', 'function': {'arguments': '{"location":"San Francisco, CA","unit":"f"}', 'name': 'getCurrentWeather'}, 'type': 'function'}]
call_y9Kp1zTC2steV8JskBTVx0QG


In [34]:
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )


run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": call_id,
        "output": "22C",
      }
    ]
)

In [35]:
wait_for_response(thread, run)

Status: completed
Elapsed time: 0 minutes 5 seconds
{
  "data": [
    {
      "id": "msg_TzgeDbdUebQ5kUKlHNKw4YjA",
      "assistant_id": "asst_MfnjYyCkMcFDUs3Fhy6KjRCf",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The current weather in San Francisco, CA is 22 degrees Celsius. If you need the temperature in Fahrenheit or more detailed weather information, please let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1709645594,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_TLq4wtYjVQwbOTXNMJJ2MnFB",
      "thread_id": "thread_MOdoVYJd9elDyIVCtogUtbnX"
    },
    {
      "id": "msg_TqDBQ9VZNG8NuvtFcRQpWyCn",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "what is the weather in San Francisco, CA?"
          },
          "type": "text"